In [435]:
import pandas as pd 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import json

In [436]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

In [437]:
# Cargar datos desde un archivo CSV
df = pd.read_excel('transporte3.xlsx')

In [438]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1458 entries, 0 to 1457
Data columns (total 11 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   Fecha                      1458 non-null   datetime64[ns]
 1   Ciudad                     1458 non-null   object        
 2   Sistema                    1458 non-null   object        
 3   Pasajeros                  1458 non-null   int64         
 4   Pasajerosdiatipicolaboral  1458 non-null   int64         
 5   Pasajerosdiasabado         1458 non-null   int64         
 6   Pasajerosdiafestivo        1458 non-null   int64         
 7   diasemana                  1458 non-null   int64         
 8   DensidadPoblacion          1458 non-null   int64         
 9   Congestion                 1458 non-null   object        
 10  Congestion2                1458 non-null   int64         
dtypes: datetime64[ns](1), int64(7), object(3)
memory usage: 125.4+ KB


In [439]:
df.head(1)

,Fecha,Ciudad,Sistema,Pasajeros,Pasajerosdiatipicolaboral,Pasajerosdiasabado,Pasajerosdiafestivo,diasemana,DensidadPoblacion,Congestion,Congestion2
0,2020-05-17,Cali,MIO,40878,436000,308387,149213,7,476878,Congestión moderada,2


In [440]:
#print(df["Pasajeros/dia"].dtype)

In [441]:
#df["Variación Transmilenio"].dtype

In [442]:
# Imprimir algunos de los valores de las características de entrenamiento
print(X_train.head())

# Imprimir algunos de los valores de la variable objetivo de entrenamiento
print(y_train.head())

      Pasajerosdiatipicolaboral  Pasajerosdiasabado  Pasajerosdiafestivo  \
254                     3860061             2499019              1188607   
1065                     128221              100186                52276   
864                     1901978             1498727               780939   
798                      144276              110000                52000   
380                     3860061             2499019              1188607   

      diasemana  DensidadPoblacion  Congestion2  Ciudad_Barranquilla  \
254           7            4102372            2                False   
1065          2             180068            2                False   
864           5            2601711            2                False   
798           4             179683            2                 True   
380           7            4126249            2                False   

      Ciudad_Bogota  Ciudad_Bucaramanga  Ciudad_Cali  ...  Ciudad_Pereira  \
254            True              

In [443]:
print(df['Pasajeros'].dtype)

int64


In [444]:
from sklearn.preprocessing import OneHotEncoder

# Crea una instancia del codificador OneHotEncoder
encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')

# Ajusta y transforma la columna Ciudad
encoded_ciudad = encoder.fit_transform(df[['Ciudad']])

# Convierte la salida en un DataFrame de pandas
encoded_ciudad_df = pd.DataFrame(encoded_ciudad, columns=encoder.get_feature_names_out(['Ciudad']))

# Concatena los DataFrames originales y codificados
df_encoded = pd.concat([df, encoded_ciudad_df], axis=1)

# Elimina la columna original 'Ciudad'
df_encoded.drop(columns=['Ciudad'], inplace=True)

In [445]:
from sklearn.preprocessing import LabelEncoder

# Crea una instancia del codificador LabelEncoder
encoder = LabelEncoder()

# Ajusta y transforma la columna Sistema
encoded_sistema = encoder.fit_transform(df['Sistema'])

# Agrega la columna codificada al DataFrame original
df['Sistema_encoded'] = encoded_sistema

In [446]:
# Define las columnas que son categóricas y numéricas
categorical_cols = ['Ciudad', 'Sistema']
numeric_cols = ['Pasajerosdiatipicolaboral', 'Pasajerosdiasabado', 'Pasajerosdiafestivo', 'diasemana']

# Crea un preprocesador para aplicar codificación One-Hot a las columnas categóricas
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(), categorical_cols),
        ('num', 'passthrough', numeric_cols)
    ])

# Combina el preprocesador con un modelo de regresión lineal en un pipeline
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', LinearRegression())
])

# Dividir los datos en características (X) y la variable objetivo (y)
X = df.drop(columns=['Pasajeros'])
y = df['Pasajeros']

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Entrenar el modelo y realizar predicciones
pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)

# Evaluar el rendimiento del modelo
mse = mean_squared_error(y_test, y_pred)
print("Error cuadrático medio:", mse)

Error cuadrático medio: 28455814587.079216


In [447]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error

# Eliminar la columna "Fecha" si no es relevante para el modelo
df = df.drop(columns=['Fecha'])

# Dividir los datos en características (X) y la variable objetivo (y)
X = df.drop(columns=['Pasajeros'])
y = df['Pasajeros']

# Convertir variables categóricas en variables numéricas utilizando One-Hot Encoding
X = pd.get_dummies(X)

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Construir y entrenar el modelo de árbol de decisión
model = DecisionTreeRegressor(random_state=42)
model.fit(X_train, y_train)

# Realizar predicciones en el conjunto de prueba
y_pred = model.predict(X_test)

# Evaluar el rendimiento del modelo
mse = mean_squared_error(y_test, y_pred)
print("Error cuadrático medio:", mse)

Error cuadrático medio: 231944701.5239726


In [448]:
import pandas as pd 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# Cargar datos desde un archivo CSV
df = pd.read_excel('transporte3.xlsx')

# Seleccionar las características relevantes para el modelo
features = ['Ciudad', 'Sistema', 'Pasajerosdiatipicolaboral', 'Pasajerosdiasabado', 'Pasajerosdiafestivo', 'diasemana', 'DensidadPoblacion', 'Congestion2']

# Dividir los datos en características (X) y la variable objetivo (y)
X = df[features]
y = df['Pasajeros']

# Convertir las variables categóricas en variables dummy utilizando one-hot encoding
X_encoded = pd.get_dummies(X)

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=42)

# Entrenar el modelo de regresión lineal
model = LinearRegression()
model.fit(X_train, y_train)

# Realizar predicciones en el conjunto de prueba
y_pred = model.predict(X_test)

# Evaluar el rendimiento del modelo utilizando el error cuadrático medio
mse = mean_squared_error(y_test, y_pred)
print("Error cuadrático medio:", mse)

Error cuadrático medio: 1.3279490009105311e-19


In [449]:
print(df.columns)

Index(['Fecha', 'Ciudad', 'Sistema', 'Pasajeros', 'Pasajerosdiatipicolaboral',
       'Pasajerosdiasabado', 'Pasajerosdiafestivo', 'diasemana',
       'DensidadPoblacion', 'Congestion', 'Congestion2'],
      dtype='object')


In [450]:
print(df.head())

       Fecha Ciudad Sistema  Pasajeros  Pasajerosdiatipicolaboral  \
0 2020-05-17   Cali     MIO      40878                     436000   
1 2020-03-25   Cali     MIO      65411                     436000   
2 2020-07-24   Cali     MIO     162505                     436000   
3 2020-05-23   Cali     MIO      97821                     436000   
4 2020-07-15   Cali     MIO     165579                     436000   

   Pasajerosdiasabado  Pasajerosdiafestivo  diasemana  DensidadPoblacion  \
0              308387               149213          7             476878   
1              308387               149213          3             501411   
2              308387               149213          5             598505   
3              308387               149213          6             533821   
4              308387               149213          3             601579   

            Congestion  Congestion2  
0  Congestión moderada            2  
1  Congestión moderada            2  
2  Congestión 